## Session 2: Sequence alignments

In this exercise worksheet, we will write our very own (rudimentary, but functional) sequence aligner. It will consist of two pieces:
1. An exact matching step to find series of short exact matches, or "seeds".
2. An inexact matching step to extend the seeds and find gapped alignments.

We will then use it to align an unknown gene from a bat coronavirus to the genome of Sars-cov-2 and identify which gene it corresponds to.

### Exercise 1: Finding exact matches

Here we will implement a suffix array to quickly lookup the positions of short sequences in a larger one.

As we saw in the lecture, there are many ways to identify exact matches between two sequences. Different methods can have different performances, in terms of speed and memory requirements. Suffix arrays allow searches in 

**a) Write the `build_suffix_array` function.**

**b) Write the `search_suffix_array` function**

**c) What is the time complexity of the building process ?**

In [1]:

def build_suffix_array(target_seq):
    """
    Given an input DNA sequence, this function should generate
    The corresponding suffix array.
    >>> build_suffix_array("ACATG")
    [0, 2, 1, 4, 3]
    """
    suffix_array = [0] * len(target_seq)

    ...
    
    return suffix_array



def search_suffix_array(target_seq, suffix_array, query_seq):
    """
    This function should return the position at which query_seq
    is found in target_seq, using the suffix array of target_seq.
    >>> search_suffix_array("ACATG", [0, 2, 1, 4, 3], "CAT")
    1
    >>> search_suffix_array("ACATG", [0, 2, 1, 4, 3], "PPP")
    -1
    """
    index = -1
    # Hint: we use binary search, but comparing strings instead of numbers
    
    ...
    
    return index

In [2]:
###########################################
### TEST YOUR CODE BY RUNNING THIS CELL ###
###########################################
assert build_suffix_array("ACATAGGGGATT") == [0, 4, 2, 9, 1, 8, 7, 6, 5, 11, 3, 10], 'The suffix array is wrong !'
assert search_suffix_array("ACATG", [0, 2, 1, 4, 3], "CAT") == 1, 'The suffix array search yields a wrong index'
print(' 0 0 0 \n0 . . 0\n0  v  0\n 0 0 0 ')
print("Congrats !!")

AssertionError: The suffix array is wrong !



**d) Can you estimate how long it would take your function to generate the suffix array for the chromosome 1 of the human genome (250Mbp) ?**
> Hint: Once you know the time complexity of `build_array`, you could use something like scipy.optimize.fit_curve() to estimate the parameters. Then you could extrapolate values from the function.

In [114]:
# No need to write any code here !
%matplotlib notebook
import matplotlib.pyplot as plt

# In this cell, we use %timeit to measure the runtime of "build_suffix_array" with different input sizes
# We then use matplotlib to visualise the results. %timeit is an iPython "magic method".
# It is available in jupyter notebooks, but not in regular python scripts.
# More about magic methods at: https://ipython.readthedocs.io/en/stable/interactive/magics.html

# We use exponentially growing sequences so that we have closely space values for short sequences
# while also measuring time for a few very long sequences

# The input sizes to test
seq_lengths = [int(1.07**i) for i in range(10, 180, 5)]
run_time = [0] * len(seq_lengths)

for i, n in enumerate(seq_lengths):
    # Note we generate a sequence of only "A". This is the worst case. Can you guess why ?
    # This means our estimates will be very pessimistic
    seq = "A" * n
    # %timeit will run each "build_suffix_array" call 3x10 times
    # It will keep the best of 3 repeats, and return the average of 20 times in it
    # Note: If this takes too much time on your computer, you can reduce -n and -r
    t = %timeit -q -n10 -r3 -o build_suffix_array(seq)
    run_time[i] = t.average

plt.plot(seq_lengths, run_time)
plt.scatter(seq_lengths, run_time)
plt.title("Run time of build_suffix_array")
plt.xlabel("Sequence length [bp]")
plt.ylabel("Run time [s]")
                    

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Run time [s]')

In [115]:
# Here you need to have figured out the time complexity of `build_array` from the code.

from scipy.optimize import curve_fit
import numpy as np
import math

def time_complexity(x, a, b):
    """Based on the number of operations, we can know the time complexity function is ..."""
    # Example if time complexity was O(n), we want a function of type f(a*x+b). The input size
    # is x, the parameters to estimate are a and b.
    # Replace the function below with your big O notation and add parameters you want to guess.
    return a * x + b #### REPLACE ME WITH THE CORRECT FUNCTION


# We can use the curve_fit function to find optimal values of the b and c parameters
# The function will automatically find a and b which minimize the following square error :
# ( run_time - time_complexity(seq_lengths, a, b) )**2
(a, b), _ = curve_fit(time_complexity, np.array(seq_lengths), np.array(run_time))

In [ ]:
# We then need to extrapolate the time_complexity to the input size of human chr1

...




### Exercise 2: Finding seeds

Many sequence alignment algorithms start by identifying regions with small exact matches to start their inexact alignments. Here we will implement a simple seeding algorithm using the suffix array above.

The target and query sequences you will be using are real biological sequences. The target is the Sars-cov-2 genome (isolate Wuhan-Hu-1). The query is a viral gene from a bat coronavirus. You will writing a home-made aligner to identify the which gene the query is.


**a) Extract k-mers from the query and feed them to the suffix array to find their position in the sequence**

**b) Merge exact matches closer than 200bp to each other into a single seed.**

**c) Extract a subsequence of 300bp around each seed. We will use them in exercise 3.**

In [117]:
from Bio import SeqIO
target = str(next(SeqIO.parse('data/session_2_target.fasta', format='fasta')).seq) # Sars-cov-2 genome
query = str(next(SeqIO.parse('data/session_2_query.fasta', format='fasta')).seq) # Unknown bat coronavirus gene

In [ ]:
def extract_kmers(seq, k=7):
    """
    Extract all unique k-mers in the input sequence.
    >>> extract_kmers("ACATGA", k=3)
    ("ACA", "CAT", "ATG", "TGA")
    """
    kmers = set()
    
    ...
    
    return kmers

def lookup_kmers(target, suffix_array, kmers):
    """
    Given a target sequence, its suffix array and a
    list of kmers (words), identify the position of
    exact matches in the target sequence for each k-mer.
    >>> lookup_kmers("ACATG", [0, 2, 1, 4, 3], ["CAT", "ATG"])
    [1, 2]
    """
    pos_seeds = []
    
    ...
    
    return pos_seeds

def merge_seeds(positions, A=100):
    """
    Given a list of sorted positions for exact matches, group all those closer than A into
    a single seed and return its middle position.
    >>> merge_seeds([0, 1, 2, 100, 103, 1000], A=10)
    [1, 101, 1000]
    """
    seeds = []
    
    ...
    
    return seeds



In [ ]:
###########################################
### TEST YOUR CODE BY RUNNING THIS CELL ###
###########################################

k_set = set(["ACA", "CAT", "ATG", "TGA"])
k_remain = set(["ACA", "CAT", "ATG", "TGA"])
for k in extract_kmers("ACATGA", k=3):
    k_remain.remove(k)
    if k not in k_set:
        raise ValueError(f"Wrong k-mer found: {k}")
if len(k_remain):
    raise ValueError(f"Missing k-mers from the extracted set: {k_remain}")
assert lookup_kmers("ACATG", [0, 2, 1, 4, 3], ["CAT", "ATG"]) == [1, 2], 'K-mers reported at wrong positions'
assert merge_seeds([0, 1, 2, 100, 103, 1000], A=10) == [1, 101, 1000], 'Seeds are not merged as expected'
print(' 0 0 0 \n0 . . 0\n0  v  0\n 0 0 0 ')
print("Congrats !!")


In [ ]:
# Now you can try running them on the viral sequences !

m, n = len(query), len(target)

### SEEDING PARAMETERS ###
# These values should work well, but you can try changing them and see how it impact the results
K = 17
A = 200
EXTEND = 300
##########################
kmers = extract_kmers(query, k=K) # Extract all unique k-mers from the query
suffix_array = build_suffix_array(target) # Build the target's suffix array
pos_kmers = lookup_kmers(target, suffix_array, kmers) # Find the position of query k-mers in target
# Remove negative values (k-mers absent from target)
pos_kmers = sorted([p for p in pos_kmers if p >= 0])
# Merge series of close exact matches into seeds
pos_seeds = merge_seeds(pos_kmers, A=A)

seq_seeds = []
print("Seed spans: ")
for pos in pos_seeds:
    start = max(0, pos - EXTEND)
    end =   min(n, pos + EXTEND)
    print(f"{start}-{end}")
    seq_seeds.append(target[start:end])

### Exercise 3

In practice, we usually want to find inexact matches. Here we will focus on finding local alignments using dynamic programming.

**a) Implement the Smith Waterson algorithm. Some of the code is already written to help you, look at the commments for hints and help.**


In [119]:
# This cell contains the meat of the inexact alignment algorithm.
# The code has been partially written, but critical parts are missing from the functions
# Try to fill up the missing parts based on what we saw in the dynamic programming lecture

def fill_score_matrix(query, target):
    '''
    Creates the dynamic-programming matrix for local sequence alignment.
    This matrix is filled with cumulative scores as the pairwise sequence
    alignment progresses.
    >>>create_score_matrix("CGT", "ACAT")
    [
     [0,0,0,0,0],
     [0,0,2,0,0],
     [0,0,0,1,0],
     [0,0,0,0,3],
    ],
    (3, 4),
    3
    '''
    # Default scores for Smith-Waterman.
    # Note we could use a substitution table instead of a fixed score for match / mismatch
    match = 2
    mismatch = -1
    gap = -1
    # We initialize a scoring matrix which we will fill using dynamic programming
    # Note the matrix has an additional row and column for the gaps (-)
    rows = len(query) + 1
    cols = len(target) + 1
    score_matrix = [[0 for col in range(cols)] for row in range(rows)]

    # Fill the scoring matrix.
    max_score = 0
    max_pos   = None    # The row and column of the highest score in matrix.
    for i in range(1, rows):
        for j in range(1, cols):
            
            ### Write your code to compute scores here ###
            
            ...
            
            ###
            score_matrix[i][j] = score

    try:
        assert max_pos is not None
    except AssertionError:
        print("No score was above zero")

    return score_matrix, max_pos, max_score


def next_move(score_matrix, x, y):
    """
    This function decides the next traceback move based
    on the score values around input position (x, y) in
    score_matrix. It will return a movement encoded as a number:
    - 0: END
    - 1: DIAG
    - 2: UP
    - 3: LEFT
    """
    # Store values of the relevant neighbours
    diag = score_matrix[x - 1][y - 1]
    up   = score_matrix[x - 1][y]
    left = score_matrix[x][y - 1]
    
    ### Write your code here to decide the next traceback move ###
    # Hint: The END move is for all relevant neighbours have a score
    # of 0. It triggers the end of the alignment
    ...
    
    ###


def traceback(query, target, score_matrix, start_pos):
    '''Find the optimal path through the matrix.

    This function traces a path from the bottom-right to the top-left corner of
    the scoring matrix. Each move corresponds to a match, mismatch, or gap in one
    or both of the sequences being aligned. Moves are determined by the score of
    three adjacent squares: the upper square, the left square, and the diagonal
    upper-left square.

    WHAT EACH MOVE REPRESENTS
        diagonal: match/mismatch
        up:       gap in sequence 1
        left:     gap in sequence 2
    
    score_matrix is the filled score matrix, start_pos is the position from which
    the traceback will start. This should be the position with the maximum score.
    '''
    
    END, DIAG, UP, LEFT = range(4)
    aligned_query = []
    aligned_target = []
    x, y = start_pos
    move = next_move(score_matrix, x, y)
    while move != END:
        if move == UP:
            aligned_query.append(query[x - 1])
            aligned_target.append('-')
            x -= 1
            
        ### Write your code for DIAG and LEFT moves
        elif move == DIAG:
            ...
        
        else:
            ...
        
        ###
        move = next_move(score_matrix, x, y)

    aligned_query.append(query[x - 1])
    aligned_target.append(target[y - 1])

    return ''.join(reversed(aligned_query)), ''.join(reversed(aligned_target))


In [ ]:
# These two functions are just for cosmetic purpose, to visualise the results of the alignment.
# You don't need to change anything here.

def alignment_string(aligned_query, aligned_target):
    '''Construct a special string showing identities, gaps, and mismatches.

    This string is printed between the two aligned sequences and shows the
    identities (|), gaps (-), and mismatches (:). As the string is constructed,
    it also counts number of identities, gaps, and mismatches and returns the
    counts along with the alignment string.

    AAGGATGCCTCAAATCGATCT-TTTTCTTGG-
    ::||::::::||:|::::::: |:  :||:|   <-- alignment string
    CTGGTACTTGCAGAGAAGGGGGTA--ATTTGG
    '''
    # Build the string as a list of characters to avoid costly string
    # concatenation.
    idents, gaps, mismatches = 0, 0, 0
    alignment_string = []
    for base_q, base_t in zip(aligned_query, aligned_target):
        if base_q == base_t:
            alignment_string.append('|')
            idents += 1
        elif '-' in (base_q, base_t):
            alignment_string.append(' ')
            gaps += 1
        else:
            alignment_string.append(':')
            mismatches += 1

    return ''.join(alignment_string), idents, gaps, mismatches

def prettify_alignment(query_aligned, target_aligned):
    # Pretty print the results. The printing follows the format of BLAST results
    # as closely as possible.
    alignment_str, idents, gaps, mismatches = alignment_string(query_aligned, target_aligned)
    alength = len(query_aligned)

    print(' Identities = {0}/{1} ({2:.1%}), Gaps = {3}/{4} ({5:.1%})'.format(idents,
          alength, idents / alength, gaps, alength, gaps / alength))
    print()
    for i in range(0, alength, 60):
        query_slice = query_aligned[i:i+60]
        print('Query  {0:<4}  {1}  {2:<4}'.format(i + 1, query_slice, i + len(query_slice)))
        print('             {0}'.format(alignment_str[i:i+60]))
        target_slice = target_aligned[i:i+60]
        print('Sbjct  {0:<4}  {1}  {2:<4}'.format(i + 1, target_slice, i + len(target_slice)))
        print()

You can use the cell below to visually test your Smith-Waterman implementation.
You should see the following output:
```
 Identities = 2/3 (66.7%), Gaps = 0/3 (0.0%)

Query  1     CGT  3   
             |:|
Sbjct  1     CAT  3   

```

In [ ]:
# Here you can give your inexact aligner a test run, you can try changing the dummy inputs 
# and see how it affects the resulting alignment
dummy_query = "GGGGGCCA"
dummy_target = "CGTTTTACAGGGGACAT"

# Initialize the scoring matrix and fill it with scores
score_matrix, start_pos, max_score = fill_score_matrix(dummy_query, dummy_target)

# Find the optimal path through the scoring matrix 
# producing the best local sequence alignment.
query_aligned, target_aligned = traceback(dummy_query, dummy_target, score_matrix, start_pos)
prettify_alignment(query_aligned, target_aligned)

**b) Run your seed sequences through the algorithm. What are the coordinates of the best the alignment ?**


In [122]:

for seed in seq_seeds:
    mat, _, score = fill_score_matrix(query, seed)
    ...


**c) What gene does that fall into ? (Look at the file "data/session_2_annotations.bed")**

**d) Can you think of a way to measure the significance of your alignments ? (discussion)**